<h1><a href='http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/'>理解NLP的卷积神经网络</a></h1>

- <a href='#convolutional_filter'>什么是卷积？</a>
- <a href='#convolutional_nn'>什么是卷积神经网络？</a>
    - <a href='#apply_nlp'>那么，CNN如何适用于NLP？</a>
- <a href='#cnn_super_arguments'>CNN超参数</a>
- <a href='#cnn_nlp'>卷积神经网络应用于NLP</a>



当我们听说卷积神经网络（CNN）时，我们通常会想到计算机视觉。

CNN负责图像分类的重大突破，是当今大多数计算机视觉系统的核心，从Facebook的自动照片标签到自动驾驶汽车。


最近，我们也开始将CNN应用于自然语言处理中的问题，并获得了一些有趣的结果。在这篇文章中，我将尝试总结：
- CNN是什么？
- CNN如何在NLP中使用？

对于计算机视觉用例来说，CNN背后的直觉有点容易理解，所以我将从那里开始，然后慢慢向NLP迈进。

<h2><a name='convolutional_filter'>什么是卷积？</a></h2>

理解卷积的最简单方法：
- 是将其视为应用于**矩阵**的**滑动窗口函数**。这是一个满口的，但看着可视化变得非常清楚：

<img src='./images/Convolution_schematic.gif' width='70%'/>

用3×3 Filter抓取特征。    
资料来源：http：//deeplearning.stanford.edu/wiki/index.php/Feature_extraction_using_convolution


想象一下，左边的矩阵代表黑白图像。每个条目对应一个像素，0表示黑色，1表示白色（对于灰度图像，通常在0到255之间）。

滑动窗口称为  **内核kernel**， **过滤器filter**或**特征检测器feature detector.**。 

在这里，我们使用3×3 Filter，将其元素值与原始矩阵相乘，然后将它们相加。  
为了获得完整的卷积，我们通过在整个矩阵上滑动 filter 来为每个元素执行此操作。



您可能想知道您实际上可以做些什么。这是一些直观的例子。

### filter应用1：使用其相邻值对每个像素求平均值会模糊图像：

<img src='./images/generic-taj-convmatrix-blur.png' width='40%'/>

### filter应用2：取像素及其邻居之间的差异来检测边缘：
为了直观地理解这一点，请考虑图像中光滑的部分会发生什么，其中像素颜色等于其邻居的颜色：添加取消，结果值为0或黑色。
如果强度有锐利边缘，从白色到黑色的过渡，例如，你得到一个很大的差异和产生的白色值

<img src='./images/adjacent_side_detector.png' width='40%'/>

在<a href='https://docs.gimp.org/en/plug-in-convmatrix.html'>GIMP手册</a>有一些其他的例子。

要了解有关卷积如何工作的更多信息，我还建议查看<a href='http://colah.github.io/posts/2014-07-Understanding-Convolutions/'>Chris Olah关于该主题的帖子。</a>

<h2><a name='convolutional_nn'>什么是卷积神经网络？</a></h2>


现在你知道什么是卷积。

但CNN呢？

CNN基本上只是几层卷积，其中**非线性激活函数**：如ReLU或tanh应用于结果。

在传统的前馈神经网络(feedforward neural network)中，我们将每个输入神经元连接到下一层中的每个输出神经元。这也称为**完全连接层(fully connected layer)** 或 **仿射层(affine layer)**。

在CNN中我们不这样做。

相反，我们在输入层上使用**卷积**来计算输出。这导致局部连接(local connections)，其中输入的每个区域连接到输出中的神经元。 每一层都应用不同的过滤器(filters)，通常是数百或数千，如上所示，并结合其结果。

还有一些叫做池(pooling)（子采样(subsampling)）层的东西，但我稍后会介绍。

在训练阶段，CNN会 根据您要执行的任务自动学习其过滤器的值。

例如，在图像分类中，CNN可以学习从第一层中的原始像素检测边缘，然后使用边缘检测第二层中的简单形状，然后使用这些形状来阻止更高级别的特征，例如：面部形状在较高层。最后一层是使用这些高级功能的分类器。

<img src='./images/cnn01.png' width='90%'/>

这个计算有两个方面值得关注：
- 位置不变性  Location Invariance    
- 组合性  Compositionality  

假设您想要对图像中是否有大象进行分类。

**位置不变性：**    
因为你在整个图像上滑动你的过滤器，你真的不关心那里的大象发生。实际上，**池化(pooling)**还可以为您提供平移translation，旋转rotation和缩放的不变性scaling，但稍后会有更多内容。

**组合性：**   
第二个关键方面是（本地）组合性。每个过滤器组成将较低级别功能的本地补丁转换为更高级别的表示。

这就是CNN在计算机视觉领域如此强大的原因。   
直观地说，您可以从像素，边缘形状和形状中更复杂的对象构建边缘。

<h3><a name='apply_nlp'>那么，CNN如何适用于NLP？</a></h3>

代替图像像素，大多数NLP任务的输入是表示为矩阵的句子或文档。

矩阵的每一行对应一个标记，通常是一个单词，但它可以是一个字符。也就是说，每行是表示单词的向量。通常，这些向量是word嵌入 （低维表示），如word2vec或GloVe，但它们也可以是将单词索引为词汇表的单热向量。

对于使用100维嵌入的10个单词的句子，我们将使用10×100矩阵作为输入。

这是我们的“图像”(image)。

在视觉中，我们的滤镜会滑过图像的局部色块，**但在NLP中，我们通常使用在矩阵的整行上滑动的滤镜（单词）。**

因此，我们的**滤波器的“宽度”通常与输入矩阵的宽度相同**。**高度或区域大小可以变化，但是通常一次滑动超过2-5个单词的窗口。**

将上述所有内容放在一起，NLP的卷积神经网络可能看起来像这样（花几分钟时间尝试理解这张图片以及如何计算维度。

您现在可以忽略池化，我们稍后会解释。

在NLP中，卷积核kernel(filter)的一般特征：
- 在矩阵上整行的滑动
- filter的“宽度”大小与矩阵的宽度相同
- filter的“高度”或区域大小可以改变，但是通常一次滑动超过2-5个单词的窗口。

<img src='./images/nlp_cnn_filter.png' width='80%'/>

图解：  
用于句子分类的卷积神经网络（CNN）架构的例证。 

这里我们描述了三个滤波器区域大小：2,3和4，每个都有2个滤波器。每个滤波器对句子矩阵执行卷积并生成（可变长度）特征映射。  
然后在每个地图上执行1-max pooling，即，the largest number from each feature map is recorded.    
因此，从所有六个图生成单变量特征向量，并且这六个特征被连接以形成倒数第二层的特征向量。  最后的softmax图层然后接收该特征向量作为输入并使用它来对句子进行分类; 这里我们假设二进制分类，因此描述了两种可能的输出状态。

那么我们对计算机视觉有什么好看的直觉呢？

**位置不变性**和**局部组合性**对图像有直观的意义，但对于NLP则不那么重要。

你可能会在句子中出现一个单词。彼此接近的像素可能在语义上相关（同一对象的一部分），但对于单词而言并非总是如此。在许多语言中，短语的一部分可以用几个其他单词分开。组成方面也不明显。很明显，单词在某些方面构成，例如：修饰名词的形容词，但这究竟是如何工作的，更高级别的表示实际上“意味着”并不像计算机视觉案例那样明显。

鉴于这一切，似乎CNN不适合NLP任务。

<a href='http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/'>递归神经网络 </a>更直观。它们类似于我们处理语言的方式（或者至少我们认为我们处理语言的方式）：从左到右顺序阅读。

幸运的是，这并不意味着CNN不起作用。  
<a href='https://en.wikipedia.org/wiki/All_models_are_wrong'>All models are wrong, but some are useful. </a>

事实证明，应用于NLP问题的CNN表现相当不错。简单的Bag of Words模型  显然过于简单化了不正确的假设，但多年来一直是标准方法，并取得了相当不错的结果。
The simple Bag of Words model is an obvious oversimplification with incorrect assumptions, but has nonetheless been the standard approach for years and lead to pretty good results.

CNN的一个重要论点是它们很快——非常快。

卷积(Convolutions )是计算机图形的核心部分，在GPU上的硬件级别上实现。与<a href='Convolutions '>n-grams</a>之类的东西相比，CNN 在表示方面也是有效的。

With a large vocabulary, computing anything more than 3-grams can quickly become expensive. Even Google doesn’t provide anything beyond 5-grams.


卷积滤波器自动学习良好的表示，无需表示整个词汇表。具有大于5的过滤器是完全合理的。我喜欢认为第一层中的许多学习过滤器捕获与n-gram非常相似（但不限于）的特征，但是以更紧凑的方式表示它们。
Convolutional Filters learn good representations automatically, without needing to represent the whole vocabulary.   
It’s completely reasonable to have filters of size larger than 5.   
I like to think that many of the learned filters in the first layer are capturing features quite similar (but not limited) to n-grams, but represent them in a more compact way.

<h2><a name='cnn_super_arguments'>CNN超参数 hyperparameters</a></h2>


在解释如何将CNN应用于NLP任务之前，让我们看一下构建CNN时需要做出的一些选择。希望这有助于您更好地了解该领域的文献。

### Narrow vs. Wide convolution 狭窄与宽的卷积

当我解释上面的卷积时，我忽略了我们如何应用滤波器的一些细节。

在矩阵的中心应用3×3滤波器工作正常，但边缘怎么样？如何将滤镜应用于顶部和左侧没有任何相邻元素的矩阵的第一个元素？
You can use zero-padding. 您可以使用零填充。  
All elements that would fall outside of the matrix are taken to be zero.落在矩阵之外的所有元素都被认为是零。  

Adding zero-padding is also called wide convolution, and not using zero-padding would be a narrow convolution. An example in 1D looks like this:

通过这样做，您可以将滤镜应用于输入矩阵的每个元素，并获得更大或相同大小的输出。   
添加零填充也称为**宽卷积(wide convolution)**，不使用零填充将是一个**窄卷积(narrow convolution)**。




下面是示例：
狭窄与广泛的卷积。滤波器大小为5，输入大小为7。

<img src='./images/narrow_wide_convolutional.png' width='80%'/>

You can see how wide convolution is useful, or even necessary, when you have a large filter relative to the input size. 

'In the above, the narrow convolution yields  an output of size (7-5) + 1=3, and a wide convolution an output of size (7+2*4 - 5) + 1 =11. More generally, the formula for the output size is $$n_{out}=(n_{in} + 2*n_{padding} - n_{filter}) + 1 $$

在上文中，窄卷积产生大小的输出（7-5）+ 1 = 3，并且宽卷积产生大小的输出（7 + 2 * 4  -  5）+ 1 = 11。更一般地说，输出大小的公式是$$n_ {out} =（n_ {in} + 2 * n_ {padding}  -  n_ {filter}）+ 1 $$

### Stride Size 步幅大小

你的卷积的另一个超参数是**stride size步幅大小**，定义你想要在每一步移动过滤器的程度。在上面的所有示例中，步幅大小为1，并且过滤器的连续应用重叠。步幅越大，滤波器的应用越少，输出尺寸越小。以下来自斯坦福大学cs231网站的  结果显示步幅大小为1和2应用于一维输入：

Another hyperparameter for your convolutions is the stride size, defining by how much you want to shift your filter at each step.  In all the examples above the stride size was 1, and consecutive applications of the filter overlapped. A larger stride size leads to fewer applications of the filter and a smaller output size. The following from the Stanford cs231 website shows stride sizes of 1 and 2 applied to a one-dimensional input:

### Pooling Layers

### Channels

<h2><a name='cnn_nlp'>卷积神经网络应用于NLP</a></h2>